# Демонстрация к четвертой задаче

## Загрузка необходимых библиотек

Для работы нам понадобятся две библиотеки:
1. `pandas`: при помощи нее мы считаем из файла таблицу с данными
2. `matplotlib`: библиотека для визуализации данных
3. `math`: библиотека математических функций
4. `sklearn`: библиотека инструментов для анализа данных и машинного обучения

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## Чтение данных из CSV файла

Таблица содержит информацию по [золоту](https://dota2-ru.gamepedia.com/%D0%97%D0%BE%D0%BB%D0%BE%D1%82%D0%BE), [опыту](https://dota2-ru.gamepedia.com/%D0%9E%D0%BF%D1%8B%D1%82) и числу добитых [крипов](https://dota2-ru.gamepedia.com/%D0%9A%D1%80%D0%B8%D0%BF%D1%8B) и прочие показатели игроков перед началом драк в матчах Dota 2.

x:
- `r1_gold`: стоимость игрока на первом слоте
- `r1_xp`: опыт игрока на первом слоте
- `r1_lh`: число добитых крипов игроком на первом слоте
- `r2_gold`: стоимость игрока на втором слоте
- `r2_xp`: опыт игрока на втором слоте
- `r2_lh`: число добитых крипов игроком на втором слоте
- `d1_gold`: стоимость игрока на шестом (первом для Dire) слоте
- `d2_gold`: стоимость игрока на седьмом (втором для Dire) слоте
...

y: команда Radiant победила в драке

In [2]:
x = pd.read_csv('data/train.csv', index_col=0)
y = pd.read_csv('data/target.csv', index_col=0)['radiant_won']

Взгляним на данные

In [3]:
x.head()

,lobby_type,r1_hero,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
fight_id,,,,,,,,,,,,,,,,,,,,,
0,0,21,22443,17670,165,13,0,30,6,23496,...,7,54.65332,NaN,NaN,400.16898,36,4,3,1,2.86596
1,0,22,18303,14005,109,9,0,38,16,15909,...,3,-23.60504,NaN,NaN,202.87660,46,5,14,11,-4.13422
2,7,76,22313,17257,247,4,0,51,98,20721,...,32,-24.89392,NaN,NaN,226.01148,46,5,19,16,-14.76307
3,7,4,15055,11879,112,11,0,27,44,16640,...,4,-0.56653,NaN,NaN,1027.51575,38,4,14,12,-21.56140
4,7,43,13003,12716,167,0,0,19,36,15265,...,1,0.29993,NaN,NaN,517.10706,38,5,2,0,475.45059


Целевая переменная

In [4]:
y.head()

fight_id
0     True
1    False
2    False
3    False
4    False
Name: radiant_won, dtype: bool

Распределение классов целевой переменной

In [5]:
y.value_counts()

True     24603
False    24485
Name: radiant_won, dtype: int64

Проверка на наличие пропусков в данных

In [6]:
count_missings = x.isnull().sum()
count_missings[count_missings > 0]

first_blood_time               49088
first_blood_team               49088
first_blood_player1            49088
first_blood_player2            49088
radiant_bottle_time            24890
radiant_courier_time           39929
radiant_flying_courier_time     2465
radiant_first_ward_time         1836
dire_bottle_time               25079
dire_courier_time              39561
dire_flying_courier_time        2541
dire_first_ward_time            2067
dtype: int64

Уберем из данных все колонки, где процент пропусков превышает порог в 50%

In [7]:
missings_threshold = x.shape[0] / 2
drop_list = count_missings[count_missings > missings_threshold].index.tolist()
print(drop_list)

x.drop(drop_list, axis=1, inplace=True)

['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'dire_bottle_time', 'dire_courier_time']


В оставшихся колонках пропуски заменим на 0

In [8]:
x = x.fillna(0)

Создадим переменные-индикаторы для категориальных признаков

In [9]:
categorical = [
    'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
    'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'
]
hero_dummies = pd.get_dummies(x[categorical[0]])
for col in categorical[1:]:
    hero_dummies = hero_dummies + pd.get_dummies(x[col])

hero_dummies.columns = ['hero_{}'.format(col_name) for col_name in hero_dummies.columns]

Заменяем категориальные колонки на переменные-индикаторы

In [10]:
x.drop(categorical, axis=1, inplace=True)
x = pd.concat([x, hero_dummies], axis=1)

Разбиваем выборку на обучающую и валидационную

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.33, random_state=1)

Оптимизация гипер-параметров модели Random Forest на кросс-валидации

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [10, 50, 100]}
clf = GridSearchCV(RandomForestClassifier(random_state=322), param_grid, verbose=3, n_jobs=-1)
clf.fit(x_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=10, score=0.5390860166012953, total=   1.8s
[CV] ......... n_estimators=10, score=0.533521846209979, total=   1.7s
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=10, score=0.5244480934136106, total=   1.9s
[CV] n_estimators=100 ................................................
[CV] ........ n_estimators=50, score=0.5528596187175043, total=   8.1s
[CV] n_estimators=100 ................................................
[CV] ........ n_estimators=50, score=0.5449238347167746, total=   7.6s
[CV] n_estimators=

[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:   10.2s remaining:    5.1s


[CV] ....... n_estimators=100, score=0.5559609595913527, total=  13.6s
[CV] ....... n_estimators=100, score=0.5481163914986774, total=  11.3s
[CV] ....... n_estimators=100, score=0.5445174238277687, total=  10.9s


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   21.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=322, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 50, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=3)

Видим, что 100 деревьев - самый лучшей вариант среди `[10, 50, 100]`

In [13]:
best_clf = clf.best_estimator_
best_clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=322, verbose=0, warm_start=False)

Оценим качество модели

In [14]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_train, best_clf.predict(x_train))
validation_acc = accuracy_score(y_validation, best_clf.predict(x_validation))
print('Train Accuracy:', train_acc)
print('Validation Accuracy:', validation_acc)

Train Accuracy: 1.0
Validation Accuracy: 0.5548765432098766
